[View in Colaboratory](https://colab.research.google.com/github/kyokicchi/kaggle_codes/blob/master/predict.ipynb)

In [1]:
!pip install lightgbm

    100% |████████████████████████████████| 1.1MB 13.7MB/s 


In [2]:
!pip install --upgrade xgboost

    100% |████████████████████████████████| 15.8MB 3.2MB/s 
  Found existing installation: xgboost 0.7.post4
    Uninstalling xgboost-0.7.post4:
      Successfully uninstalled xgboost-0.7.post4


In [3]:
from google.colab import drive
from google.colab import files

import pickle

import numpy as np
import pandas as pd

import xgboost as xgb
import lightgbm as lgb
from keras import models
from keras import layers
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn.metrics import mean_squared_error


Using TensorFlow backend.


In [4]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [5]:
%cd ../content/gdrive/My Drive/projects/

/content/gdrive/My Drive/projects


In [6]:
%ls

model_LGB.sav  model_XGB.sav  user_test_n.csv
model_NN.h5    output.csv     user_train_n.csv


In [7]:
%%time
df_user_train = pd.read_csv('user_train_n.csv', index_col = 0, dtype={'fullVisitorId': 'str'})
df_user_test = pd.read_csv('user_test_n.csv', index_col = 0, dtype={'fullVisitorId': 'str'})

df_y = df_user_train['transactionRevenue_sum']
df_x = df_user_train.drop(['transactionRevenue_sum'], axis=1)
df_tgt = df_user_test.drop(['transactionRevenue_sum'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, random_state=1)
x_eval, x_valid, y_eval, y_valid = train_test_split(x_test, y_test, random_state=1)

CPU times: user 18.2 s, sys: 2.14 s, total: 20.4 s
Wall time: 25.6 s


In [17]:
%%time
models = []

filename = 'model_LGB.sav'
model_LGB = pickle.load(open(filename, 'rb'))
models.append(model_LGB)

filename = 'model_XGB.sav'
model_XGB = pickle.load(open(filename, 'rb'))
models.append(model_XGB)

#filename = 'model_NN.h5'
#model_NN = load_model(filename)
#models.append(model_NN)



CPU times: user 29.3 ms, sys: 3.03 ms, total: 32.4 ms
Wall time: 31.1 ms


In [18]:
for model in models:
    y_pred = model.predict(x_valid.fillna(0))
    mse = mean_squared_error(y_valid, y_pred)
    rmse = np.sqrt(mse)
    print(rmse)

0.003357617885289748
0.0032958225964899368


In [19]:
%%time
preds = []
for model in models:
    pred_tmp = model.predict(df_tgt)
    preds.append(pred_tmp)

CPU times: user 19.5 s, sys: 70 ms, total: 19.6 s
Wall time: 10.3 s


In [20]:
%%time
df_preds = pd.DataFrame(preds).T

CPU times: user 18.8 s, sys: 513 ms, total: 19.3 s
Wall time: 19.4 s


In [21]:
df_preds.describe()

,0,1
count,617242.000000,617242.000000
mean,0.000379,0.000412
std,0.002760,0.002753
min,-0.017943,-0.008842
25%,0.000003,0.000018
50%,0.000003,0.000018
75%,0.000003,0.000018
max,0.136647,0.529992


In [0]:
pred_tgt = df_preds[df_preds.columns].sum(axis=1)/len(preds)
pred_tgt[pred_tgt<0] = 0
pred_tgt = pred_tgt.values.tolist()

In [25]:
pred_tgt[:5]

[0.00018983326278793912,
 1.8020066770766976e-05,
 1.0569486173843148e-05,
 1.8020066770766976e-05,
 1.0569486173843148e-05]

In [0]:
df_tgt['PredictedLogRevenue'] = pred_tgt

In [0]:
df_tgt[['PredictedLogRevenue']].to_csv("output.csv", index=True)

In [28]:
df_output = pd.read_csv('output.csv')
df_output.head()

,fullVisitorId,PredictedLogRevenue
0,0000000259678714014,0.000190
1,0000049363351866189,0.000018
2,0000053049821714864,0.000011
3,0000059488412965267,0.000018
4,0000085840370633780,0.000011


In [29]:
type(df_output['fullVisitorId'][0])

str

In [30]:
df_output.shape

(617242, 2)